In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 512
test_batch = 500
lr = 0.04
schedule = [75, 150, 225]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/64/b1' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (64, 64)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 300] LR: 0.040000
1/251 Data:12.951 | Batch:33.328 | Total:0:00:33 | ETA:2:18:52 | Loss:0.7040129899978638 | top1:46.484375
11/251 Data:0.067 | Batch:2.445 | Total:0:00:56 | ETA:0:09:24 | Loss:0.7121596986597235 | top1:50.408382415771484
21/251 Data:0.002 | Batch:0.783 | Total:0:01:06 | ETA:0:04:32 | Loss:0.7123707674798512 | top1:50.939361572265625
31/251 Data:0.002 | Batch:0.700 | Total:0:01:13 | ETA:0:02:37 | Loss:0.7205051433655524 | top1:50.66154098510742
41/251 Data:0.020 | Batch:2.142 | Total:0:01:33 | ETA:0:06:58 | Loss:0.7186140577967574 | top1:50.595462799072266
51/251 Data:0.004 | Batch:1.697 | Total:0:01:55 | ETA:0:07:16 | Loss:0.7147336637272554 | top1:50.52849578857422
61/251 Data:0.001 | Batch:2.432 | Total:0:02:22 | ETA:0:08:30 | Loss:0.7120178449349325 | top1:50.60194396972656
71/251 Data:0.001 | Batch:0.920 | Total:0:02:39 | ETA:0:05:43 | Loss:0.7118673618410675 | top1:50.51441192626953
81/251 Data:0.001 | Batch:1.648 | Total:0:02:54 | ETA:0:04:09 | Loss:

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


65/65 Data:0.001 | Batch:1.451 | Total:0:01:15 | ETA:0:00:00 | Loss:0.7000923587525745 | top1:50.043617248535156

Epoch: [2 | 300] LR: 0.068000
1/251 Data:7.317 | Batch:9.179 | Total:0:00:09 | ETA:0:38:15 | Loss:0.717559814453125 | top1:52.734375
11/251 Data:0.023 | Batch:1.671 | Total:0:00:30 | ETA:0:08:31 | Loss:0.7360568913546476 | top1:49.78693389892578
21/251 Data:0.022 | Batch:2.746 | Total:0:00:54 | ETA:0:09:09 | Loss:0.7419278820355734 | top1:49.8046875
31/251 Data:0.001 | Batch:2.563 | Total:0:01:19 | ETA:0:09:08 | Loss:0.7328598460843486 | top1:50.277217864990234
41/251 Data:0.048 | Batch:2.819 | Total:0:01:45 | ETA:0:09:13 | Loss:0.7268638421849507 | top1:50.452552795410156
51/251 Data:0.051 | Batch:2.752 | Total:0:02:07 | ETA:0:07:12 | Loss:0.7240841108209947 | top1:50.45956039428711
61/251 Data:0.002 | Batch:1.116 | Total:0:02:22 | ETA:0:04:58 | Loss:0.7197847981921962 | top1:50.9893684387207
71/251 Data:0.145 | Batch:2.405 | Total:0:02:40 | ETA:0:05:12 | Loss:0.7169299696

191/251 Data:0.001 | Batch:2.051 | Total:0:06:30 | ETA:0:02:40 | Loss:0.6550749680758771 | top1:61.2084846496582
201/251 Data:0.010 | Batch:3.058 | Total:0:06:57 | ETA:0:02:13 | Loss:0.6544601398320934 | top1:61.320350646972656
211/251 Data:0.001 | Batch:1.151 | Total:0:07:25 | ETA:0:01:53 | Loss:0.6539094411366358 | top1:61.416988372802734
221/251 Data:0.001 | Batch:2.345 | Total:0:07:49 | ETA:0:01:12 | Loss:0.6538787376287296 | top1:61.3944091796875
231/251 Data:0.001 | Batch:1.706 | Total:0:08:13 | ETA:0:00:50 | Loss:0.6540437389761855 | top1:61.37462615966797
241/251 Data:0.020 | Batch:1.432 | Total:0:08:33 | ETA:0:00:20 | Loss:0.6543622313693351 | top1:61.324886322021484
65/65 Data:0.000 | Batch:0.757 | Total:0:02:10 | ETA:0:00:00 | Loss:0.5100135858927932 | top1:78.92523956298828

Epoch: [5 | 300] LR: 0.152000
1/251 Data:14.517 | Batch:16.895 | Total:0:00:16 | ETA:1:10:24 | Loss:0.639108419418335 | top1:65.0390625
11/251 Data:0.001 | Batch:2.526 | Total:0:00:43 | ETA:0:10:33 | Lo

131/251 Data:0.001 | Batch:1.888 | Total:0:04:18 | ETA:0:05:02 | Loss:0.6175643073693486 | top1:65.93660736083984
141/251 Data:0.051 | Batch:3.509 | Total:0:04:49 | ETA:0:05:43 | Loss:0.6165110181409417 | top1:66.1305923461914
151/251 Data:0.006 | Batch:1.572 | Total:0:05:10 | ETA:0:03:28 | Loss:0.6155552785128158 | top1:66.23551177978516
161/251 Data:0.001 | Batch:4.490 | Total:0:05:38 | ETA:0:04:17 | Loss:0.6146543844146017 | top1:66.3322525024414
171/251 Data:0.003 | Batch:1.464 | Total:0:06:03 | ETA:0:03:20 | Loss:0.613764662840213 | top1:66.4279556274414
181/251 Data:0.011 | Batch:2.108 | Total:0:06:21 | ETA:0:02:07 | Loss:0.6131420020240447 | top1:66.51847839355469
191/251 Data:0.027 | Batch:2.188 | Total:0:06:49 | ETA:0:02:45 | Loss:0.6120266914367676 | top1:66.63837432861328
201/251 Data:0.112 | Batch:2.593 | Total:0:07:14 | ETA:0:02:05 | Loss:0.6118496997439447 | top1:66.59378814697266
211/251 Data:0.001 | Batch:2.433 | Total:0:07:35 | ETA:0:01:28 | Loss:0.6112402621038717 | t

...

71/251 Data:0.001 | Batch:0.981 | Total:0:02:58 | ETA:0:03:39 | Loss:0.40581257956128725 | top1:81.0024185180664
81/251 Data:0.016 | Batch:1.348 | Total:0:03:13 | ETA:0:04:20 | Loss:0.4056046660299654 | top1:80.98235321044922
91/251 Data:0.019 | Batch:1.165 | Total:0:03:28 | ETA:0:03:58 | Loss:0.4067852647749932 | top1:80.88298797607422
101/251 Data:0.106 | Batch:1.582 | Total:0:03:41 | ETA:0:03:05 | Loss:0.40856886677222676 | top1:80.76654815673828
111/251 Data:0.001 | Batch:1.181 | Total:0:03:55 | ETA:0:03:21 | Loss:0.4079227270306768 | top1:80.79075622558594
121/251 Data:0.016 | Batch:1.204 | Total:0:04:13 | ETA:0:03:50 | Loss:0.4077879131332902 | top1:80.79964447021484
131/251 Data:0.007 | Batch:1.151 | Total:0:04:26 | ETA:0:02:48 | Loss:0.4077701081756417 | top1:80.83104705810547
141/251 Data:0.053 | Batch:1.348 | Total:0:04:39 | ETA:0:02:16 | Loss:0.4076775437128459 | top1:80.83998107910156
151/251 Data:0.021 | Batch:2.468 | Total:0:04:56 | ETA:0:02:50 | Loss:0.4081516532313745 |

11/251 Data:0.024 | Batch:2.462 | Total:0:00:40 | ETA:0:08:59 | Loss:0.4101325544443997 | top1:80.9303970336914
21/251 Data:0.016 | Batch:2.045 | Total:0:00:58 | ETA:0:06:46 | Loss:0.4101075799692245 | top1:80.56175994873047
31/251 Data:0.001 | Batch:1.423 | Total:0:01:15 | ETA:0:06:25 | Loss:0.40756612727718966 | top1:80.8719711303711
41/251 Data:0.006 | Batch:2.062 | Total:0:01:29 | ETA:0:04:54 | Loss:0.40815313051386576 | top1:80.74028015136719
51/251 Data:0.008 | Batch:1.720 | Total:0:01:55 | ETA:0:08:33 | Loss:0.408914784006044 | top1:80.59896087646484
61/251 Data:0.005 | Batch:1.190 | Total:0:02:22 | ETA:0:08:34 | Loss:0.4109981206596875 | top1:80.46234130859375
71/251 Data:0.056 | Batch:2.304 | Total:0:02:40 | ETA:0:05:20 | Loss:0.41214307051309396 | top1:80.38622283935547
81/251 Data:0.080 | Batch:1.438 | Total:0:02:56 | ETA:0:04:42 | Loss:0.41389471218909746 | top1:80.3144302368164
91/251 Data:0.001 | Batch:1.178 | Total:0:03:10 | ETA:0:03:44 | Loss:0.41346411619867596 | top1:

211/251 Data:0.020 | Batch:2.527 | Total:0:08:28 | ETA:0:01:41 | Loss:0.4035417399417733 | top1:80.92972564697266
221/251 Data:0.085 | Batch:1.880 | Total:0:08:51 | ETA:0:01:09 | Loss:0.40350141312202176 | top1:80.91947174072266
231/251 Data:0.038 | Batch:2.971 | Total:0:09:16 | ETA:0:00:49 | Loss:0.40398547530690315 | top1:80.87544250488281
241/251 Data:0.001 | Batch:2.391 | Total:0:09:42 | ETA:0:00:27 | Loss:0.40399145387514995 | top1:80.85856628417969
65/65 Data:0.000 | Batch:0.656 | Total:0:01:34 | ETA:0:00:00 | Loss:0.18477172300080272 | top1:99.74143981933594

Epoch: [52 | 300] LR: 0.306167
1/251 Data:17.284 | Batch:19.811 | Total:0:00:19 | ETA:1:22:33 | Loss:0.3843940794467926 | top1:83.59375
11/251 Data:0.011 | Batch:0.692 | Total:0:00:34 | ETA:0:13:23 | Loss:0.4096227450804277 | top1:80.61079406738281
21/251 Data:0.006 | Batch:1.802 | Total:0:00:49 | ETA:0:05:57 | Loss:0.4097522170770736 | top1:80.56175994873047
31/251 Data:0.001 | Batch:1.265 | Total:0:01:04 | ETA:0:05:20 | L

151/251 Data:0.036 | Batch:3.141 | Total:0:05:06 | ETA:0:03:33 | Loss:0.40162904807273914 | top1:80.85161590576172
161/251 Data:0.001 | Batch:2.772 | Total:0:05:30 | ETA:0:03:37 | Loss:0.40175600840438225 | top1:80.81449127197266
171/251 Data:0.001 | Batch:1.749 | Total:0:05:58 | ETA:0:03:46 | Loss:0.4020851221349504 | top1:80.79084777832031
181/251 Data:0.001 | Batch:2.800 | Total:0:06:24 | ETA:0:03:02 | Loss:0.40182065930814376 | top1:80.79571533203125
191/251 Data:0.001 | Batch:3.789 | Total:0:06:53 | ETA:0:02:50 | Loss:0.40191312204480795 | top1:80.81029510498047
201/251 Data:0.001 | Batch:3.563 | Total:0:07:17 | ETA:0:02:04 | Loss:0.4023091407557625 | top1:80.7864990234375
211/251 Data:0.001 | Batch:2.051 | Total:0:07:42 | ETA:0:01:39 | Loss:0.40209008435502436 | top1:80.82512664794922
221/251 Data:0.022 | Batch:1.707 | Total:0:08:08 | ETA:0:01:20 | Loss:0.40166044788123256 | top1:80.8673324584961
231/251 Data:0.001 | Batch:2.940 | Total:0:08:35 | ETA:0:00:55 | Loss:0.401388245098

91/251 Data:0.013 | Batch:1.893 | Total:0:04:12 | ETA:0:06:55 | Loss:0.3984874244574662 | top1:80.79713439941406
101/251 Data:0.001 | Batch:3.112 | Total:0:04:39 | ETA:0:06:38 | Loss:0.39888981102716803 | top1:80.7762222290039
111/251 Data:0.002 | Batch:0.806 | Total:0:05:00 | ETA:0:05:32 | Loss:0.39809006425711485 | top1:80.7872314453125
121/251 Data:0.001 | Batch:2.256 | Total:0:05:23 | ETA:0:04:59 | Loss:0.39760905652006795 | top1:80.8319320678711
131/251 Data:0.025 | Batch:2.699 | Total:0:05:44 | ETA:0:04:11 | Loss:0.39760711220384554 | top1:80.82657623291016
141/251 Data:0.007 | Batch:0.876 | Total:0:06:01 | ETA:0:03:01 | Loss:0.39631788400893514 | top1:80.89261627197266
151/251 Data:0.018 | Batch:1.269 | Total:0:06:14 | ETA:0:02:16 | Loss:0.3962106527082178 | top1:80.920166015625
161/251 Data:0.001 | Batch:2.480 | Total:0:06:40 | ETA:0:03:52 | Loss:0.396736702008277 | top1:80.8436050415039
171/251 Data:0.001 | Batch:2.676 | Total:0:07:07 | ETA:0:03:41 | Loss:0.39611566031885426 |